In [1]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import os
import copy
import cv2
import json
import transformers
from transformers import AutoImageProcessor, ViTModel
from PIL import Image, ImageMath
from transformers import AutoModel
import torch.nn as nn
from transformers import AutoTokenizer, BertModel, BertTokenizer
from torch.optim import Adam
from tqdm import tqdm
import torch.nn.functional as F
from ast import literal_eval
from torchvision.models import resnet50,SwinTransformer
import torchvision
from transformers import AutoFeatureExtractor, ResNetForImageClassification
from transformers import AutoImageProcessor, AutoModelForImageClassification
import requests
from io import BytesIO
from transformers import RobertaTokenizer, RobertaModel
from focal_loss.focal_loss import FocalLoss
from sklearn.metrics import classification_report
# from focal_loss_pytorch.focal_loss_pytorch.focal_loss import BinaryFocalLoss
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/server-ailab-12gb/miniconda3/envs/Khoa_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
version = "T_W_nondropout_FOCAL_2_03_train_VIT_BERT"

In [3]:
FOLDER = "/home/server-ailab-12gb/DUC-MMM"
data_dir = '/home/server-ailab-12gb/DUC-MMM/data/Musti'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
train_img_dir = os.path.join(train_dir, 'img')
train_file_path = os.path.join(train_dir, 'train.json')

test_img_dir = os.path.join(test_dir, 'img')
test_file_path = os.path.join(test_dir, 'test.json')
history_save_folder = os.path.join(FOLDER, 'history_save')
checkpoint_dir = os.path.join(FOLDER, 'checkpoint')
checkpoint_directory = os.path.join(checkpoint_dir, version)
CHECKPOINT_EXTENSION = '.pt'

predict_dir = os.path.join(FOLDER, "predict")
predict_path = os.path.join(predict_dir, version + '.json')
report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)

report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)

train_index_checkpoint = ''
test_index_checkpoint = ''
lr = 0.001
epoch = 0
loss = 10000

In [4]:
def save_checkpoint(checkpoint_directory, epoch, model, optimizer, LOSS, checkpoint_name=None):
    """
    The checkpoint will be saved in `checkpoint_directory` with name `checkpoint_name`.
    If `checkpoint_name` is None, the checkpoint will be saved with name `next_checkpoint_name_id + epoch`.
    """
    if checkpoint_directory is not None:
        if checkpoint_name is None:
            checkpoint_name = f'{epoch}{CHECKPOINT_EXTENSION}'

        path = os.path.join(checkpoint_directory, checkpoint_name)
        print("path: "+str(path))
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': LOSS,
                    }, path)
def load_checkpoint(PATH, model, optimizer):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return epoch, model, optimizer, loss

def save_report(pred_list, target_list, report_path):
    report = classification_report(target_list, pred_list, target_names=['NO', 'YES'], digits=6)
    with open(report_path, 'w') as f:
        f.write(report)
        
def read_json(parient_dir, name=None):
    # Opening JSON file
    if name == None:
        path = parient_dir
    else:
        path = os.path.join(parient_dir, str(name) + ".json")
    with open(path, 'r') as openfile:

        # Reading from json file
        json_object = json.load(openfile)
    return json_object
def write_json(dict, parient_dir, name=None):
    # Serializing json
    json_object = json.dumps(dict, indent=4)
    if name != None:
        path = os.path.join(parient_dir, str(name)+".json")
    else:
        path = parient_dir
    # Writing to sample.json
    with open(path, "w") as outfile:
        outfile.write(json_object)
# img_dir = '/content/drive/MyDrive/HaruLab/Dataset/Musti/test/img'
# json_file =
# values = json_file['pairs']

def load_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        # Open the image using Pillow
        image = Image.open(BytesIO(response.content))
        return image
    except requests.exceptions.RequestException as e:
        print(f"Error loading image from URL: {e}")
        return None
def split_filename(filepath):
    # Split the filepath into the directory and file parts
    directory, full_filename = os.path.split(filepath)

    # Split the file part into the base name and extension
    base_name, file_extension = os.path.splitext(full_filename)

    return directory, base_name, file_extension

## . Dataset&DataLoader

In [5]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# tokenizer = AutoModel.from_pretrained("google/mt5-base")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
vit_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
resnet_encoder = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
resnet_encoder = nn.Sequential(*list(resnet_encoder.children())[:-1])
swintransformer = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
swintransformer = nn.Sequential(*list(swintransformer.children())[:-1])
# text_encoder = BertModel.from_pretrained("google/mt5-base").to(device)
text_encoder = BertModel.from_pretrained('bert-base-multilingual-cased')
encoder_layer = text_encoder.encoder
text_embedding= text_encoder.embeddings
pooling_layer= text_encoder.pooler

In [6]:
class Dataset_MMM(Dataset):
    def __init__(self, file_path,image_dir ,image_processor,tokenizer):
        super(Dataset_MMM, self).__init__()
        self.image_dir = image_dir
        data = read_json(file_path)
        if isinstance(data, dict):
            data = data['pairs']
        print(self.image_dir)
        self.data = []
        for obj in data:
          if os.path.exists(os.path.join(self.image_dir, obj['image'].rstrip())):
            self.data.append(obj)
          else:
            _, base_name, file_extension = split_filename(obj['image'].rstrip())
            if os.path.exists(os.path.join(self.image_dir, base_name + file_extension)):
              obj['image'] = base_name + file_extension
              self.data.append(obj)
            else:
              img = load_image_from_url(obj['image'].rstrip())
              obj['image'] = base_name + file_extension
              img.save(os.path.join(self.image_dir, base_name + file_extension))
              
              self.data.append(obj)
              print(base_name + file_extension)
        print("Len: " +str(len(self.data)))
        self.image_processor = image_processor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir,self.data[index]['image'])
        text = self.data[index]['text']
        target = self.data[index]['subtask1_label']
        if target == 'NO':
          target = 0
        else:
          target = 1
        # process_image
        img = Image.open(img_path)
        if img.mode == 'L':
          img = img.convert('RGB')
        img = self.image_processor(images = img, return_tensors="pt",padding=True)
        # process_text
        text = self.tokenizer(text,padding="max_length", max_length=197,truncation=True, return_tensors="pt")

        return img, text, target

## Model

### Image

In [7]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
early_stopper = EarlyStopper(patience=5, min_delta=0)

### Text

### CrossModel

In [8]:
class CrossModel(nn.Module):
  def __init__(self,image_encoder,text_encoder,text_embedding, pooling_layer):
    super(CrossModel,self).__init__()
    #image_encoder
    self.vit_encoder = image_encoder[0]
    self.resnet_encoder = image_encoder[1]
    self.swintransformer_encoder = image_encoder[2]

    #text_encoder & text embedding
    self.text_encoder = text_encoder
    self.text_embedding = text_embedding
    self.pooling_layer = pooling_layer

    # layernorm
    self.layernorm_197 = nn.LayerNorm(197)
    self.layernorm_768 = nn.LayerNorm(768)

    # self.layernorm_1024 = nn.LayerNorm(1024)
    #linear
    self.linear = nn.Linear(2,1)

    # projection
    # self.resnet_projection_2 = nn.Linear(2048,197)
    self.resnet_projection_2 = nn.Linear(2048,197)
    self.resnet_projection_3 = nn.Linear(49,768)
    
    # Dropout
    self.dropout = nn.Dropout(p=0.2)

    self.swintransformer_projection_1 = nn.Linear(49,197)
    self.final_projection = nn.Linear(197,1)
    self.voting = nn.Linear(3,1)

  def forward(self,img,text):
    batch_size = img.size(0)
    #text_features
    text_embedding = self.text_embedding(text) # torch.Size([16, 197, 1024])

    #image_features
    image_features_0 = self.vit_encoder(img).last_hidden_state #torch.Size([16, 197, 768])
    image_features_1 = self.resnet_encoder(img).last_hidden_state #torch.Size([16, 512, 7, 7])
    image_features_2 = self.swintransformer_encoder(img).last_hidden_state #torch.Size([16, 49, 768])
    dim = image_features_1.size(1)
    #projection_1
    image_features_1 = image_features_1.reshape(batch_size, dim,-1) #torch.Size([16, 512, 49])
    image_features_1 = image_features_1.permute(0,2,1) #torch.Size([16, 49, 512])
    image_features_1 = self.resnet_projection_2(image_features_1) #torch.Size([16, 49, 197])
    image_features_1 = self.layernorm_197(image_features_1)
    image_features_1 = image_features_1.permute(0,2,1) #torch.Size([16, 197, 49])
    image_features_1 = self.resnet_projection_3(image_features_1) #torch.Size([16, 197, 768])
    image_features_1 = self.layernorm_768(image_features_1)
    # image_features_1 = self.dropout(image_features_1)
    #projection_2
    image_features_2 = image_features_2.permute(0,2,1) #torch.Size([16, 768, 49])
    image_features_2 = self.swintransformer_projection_1(image_features_2) #torch.Size([16, 768, 197])
    image_features_2 = self.layernorm_197(image_features_2)
    image_features_2 = image_features_2.permute(0,2,1) #torch.Size([16, 197, 768])

    final_features_0 = self.layernorm_768(self.linear(torch.cat([image_features_0.unsqueeze(-1) , text_embedding.unsqueeze(-1)], dim =-1)).squeeze(-1) + text_embedding)
    final_features_1 = self.layernorm_768(self.linear(torch.cat([image_features_1.unsqueeze(-1) , text_embedding.unsqueeze(-1)], dim =-1)).squeeze(-1) + text_embedding)
    final_features_2 = self.layernorm_768(self.linear(torch.cat([image_features_2.unsqueeze(-1) , text_embedding.unsqueeze(-1)], dim =-1)).squeeze(-1) + text_embedding)

    final_features_0 = self.pooling_layer(self.text_encoder(final_features_0).last_hidden_state)
    final_features_1 = self.pooling_layer(self.text_encoder(final_features_1).last_hidden_state)
    final_features_2 = self.pooling_layer(self.text_encoder(final_features_2).last_hidden_state)


    image_features_0 = self.final_projection(image_features_0.permute(0,2,1)).squeeze(-1)
    image_features_1 = self.final_projection(image_features_1.permute(0,2,1)).squeeze(-1)
    image_features_2 = self.final_projection(image_features_2.permute(0,2,1)).squeeze(-1)

    logits_0 = F.cosine_similarity(final_features_0,image_features_0, dim =-1)
    logits_1 = F.cosine_similarity(final_features_1,image_features_1, dim =-1)
    logits_2 = F.cosine_similarity(final_features_2,image_features_2, dim =-1)
    logits = self.voting(torch.cat([logits_0.unsqueeze(-1), logits_1.unsqueeze(-1), logits_2.unsqueeze(-1)], dim =-1)).squeeze(-1)
    return logits

## Train

In [9]:
# dataset_train = Dataset_MMM(train_file_path, train_img_dir,image_processor,tokenizer,is_url=is_url)
# dataloader_train = DataLoader(dataset_train,batch_size=20,drop_last=True, shuffle=True)
# len(dataset_train)

In [10]:
def train(train_loader, model,device, epochs=10, total_iterations_limit=None, optimizer=None, cur_epoch=0, best_valid_loss=10000):
    # model = model.to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    history = {"loss": [],
                "acc": []}
    

    total_iterations = 0
    loss_fn = FocalLoss(gamma=2, weights=torch.tensor(0.3))
    for epoch in range(cur_epoch + 1, epochs):
        model.train()
        avg_acc = 0
        acc_sum = 0
        avg_loss = 0
        loss_sum = 0
        num_iterations = 0
        pred_list = []
        target_list = []

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            img = torch.squeeze(data[0]['pixel_values'],1)
            text = torch.squeeze(data[1]['input_ids'],1)
            target = torch.squeeze(data[2])
            # print(text.shape)
            img = img.to(device)
            text = text.to(device)
            target = target.to(device).to(torch.int64)
            optimizer.zero_grad()
            
            logits = model(img,text)
            loss = loss_fn(torch.sigmoid(logits), target)
            loss_sum += loss.item()

            pred = torch.round(torch.sigmoid(logits.detach()))
            correct_pred = (target == pred).float()
            acc_sum += (correct_pred.sum() / len(correct_pred)).cpu().item()
            

            pred_list.extend(pred.cpu().numpy())
            target_list.extend(target.cpu().numpy())


            data_iterator.set_postfix(loss=loss_sum / num_iterations)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return

        avg_loss = loss_sum / num_iterations
        avg_acc = acc_sum / num_iterations

        if avg_loss < best_valid_loss:
            best_valid_loss = avg_loss
            save_checkpoint(checkpoint_directory, epoch, model, optimizer, avg_loss)
            save_report(pred_list=pred_list, target_list=target_list, report_path= os.path.join(report_path, str(epoch) + '.txt'))
        history['loss'].append(avg_loss)
        history['acc'].append(avg_acc)
        print("Accuracy: "+str(avg_acc))
        if early_stopper.early_stop(avg_loss):
                print("Early stop at epoch: "+str(epoch) + " with valid loss: "+str(avg_loss))
                break
    return history

def predict(model,device,dataloader,image_processor):
  list = []
  model.eval()

  data_iterator = tqdm(dataloader, desc=f'Evaluate')
  with torch.no_grad():
    for data in data_iterator:
      img = torch.squeeze(data[0]['pixel_values'],1)
      text = torch.squeeze(data[1]['input_ids'],1)
      target = torch.squeeze(data[2])
      # print(text.shape)
      img = img.to(device)
      text = text.to(device)
      target = target.to(device).to(torch.int64)
      
      logits = model(img,text)

      print("logits"+str(torch.sigmoid(logits)))
      pred = torch.round(torch.sigmoid(logits)).detach().cpu().numpy()
      list.extend(pred)
  return list

In [11]:
dataset_train = Dataset_MMM(train_file_path, train_img_dir,image_processor,tokenizer)
dataloader_train = DataLoader(dataset_train,batch_size=20,drop_last=True, shuffle=True)

/home/server-ailab-12gb/DUC-MMM/data/Musti/train/img
Len: 2374


In [12]:
img_encoder =[vit_encoder,resnet_encoder,swintransformer]
model = CrossModel(img_encoder,encoder_layer,text_embedding, pooling_layer)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

if len(train_index_checkpoint) > 0:
    PATH = os.path.join(checkpoint_directory, train_index_checkpoint + CHECKPOINT_EXTENSION)
    print("Load checkpoint: "+str(PATH))
    epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)

history_losses = train(dataloader_train,model,device ,epochs=100, optimizer=optimizer, cur_epoch=epoch, best_valid_loss=loss)
write_json(history_losses, parient_dir=history_save_folder, name=version)

Epoch 1: 100%|██████████| 118/118 [01:51<00:00,  1.06it/s, loss=0.144]


path: /home/server-ailab-12gb/DUC-MMM/checkpoint/T_W_nondropout_FOCAL_2_03_train_VIT_BERT/1.pt
Accuracy: 0.7326271287718061


Epoch 2: 100%|██████████| 118/118 [01:52<00:00,  1.05it/s, loss=0.13] 


path: /home/server-ailab-12gb/DUC-MMM/checkpoint/T_W_nondropout_FOCAL_2_03_train_VIT_BERT/2.pt
Accuracy: 0.7601695035473776


Epoch 3: 100%|██████████| 118/118 [01:52<00:00,  1.04it/s, loss=0.122]


...

Epoch 22: 100%|██████████| 118/118 [01:44<00:00,  1.13it/s, loss=0.0882]


Accuracy: 0.8300847633410309


Epoch 23: 100%|██████████| 118/118 [01:44<00:00,  1.13it/s, loss=0.0864]


Accuracy: 0.8372881508479684
Early stop at epoch: 23 with valid loss: 0.08644674266925302


FileNotFoundError: [Errno 2] No such file or directory: '/home/server-ailab-12gb/DUC-MMM/history_save/T_W_nondropout_FOCAL_2_03_train_VIT_BERT.json'

In [13]:
dataset_test = Dataset_MMM(test_file_path, test_img_dir,image_processor,tokenizer)
dataloader_test = DataLoader(dataset_test,batch_size=16,drop_last=False, shuffle=False)

/home/server-ailab-12gb/DUC-MMM/data/Musti/test/img
Len: 814


In [14]:
if len(test_index_checkpoint) > 0:
    img_encoder =[vit_encoder,resnet_encoder,swintransformer]
    model = CrossModel(img_encoder,encoder_layer,text_embedding, pooling_layer)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    PATH = os.path.join(checkpoint_directory, test_index_checkpoint + CHECKPOINT_EXTENSION)
    epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)
    print("Load checkpoint: "+str(PATH))
list = predict(model,device,dataloader_test,image_processor)
json_dict = read_json(test_file_path)
for i in range(len(json_dict['pairs'])):
    if list[i] == 0:
        json_dict['pairs'][i]['subtask1_label'] = 'NO'
    else:
        json_dict['pairs'][i]['subtask1_label'] = 'YES'
with open(predict_path, 'w') as f:
    json.dump(json_dict, f, indent=4)

Evaluate:   2%|▏         | 1/51 [00:00<00:17,  2.78it/s]

logitstensor([0.1730, 0.1602, 0.4063, 0.4766, 0.1863, 0.2823, 0.1747, 0.1730, 0.1722,
        0.1722, 0.5204, 0.4470, 0.1578, 0.3973, 0.1710, 0.5559],
       device='cuda:0')


Evaluate:   4%|▍         | 2/51 [00:00<00:16,  2.95it/s]

logitstensor([0.6488, 0.1602, 0.1649, 0.4880, 0.3651, 0.1724, 0.5466, 0.1609, 0.1806,
        0.1610, 0.1610, 0.6052, 0.5515, 0.1722, 0.5559, 0.1793],
       device='cuda:0')


Evaluate:   6%|▌         | 3/51 [00:01<00:15,  3.00it/s]

...

logitstensor([0.2358, 0.3433, 0.6052, 0.2102, 0.2014, 0.5963, 0.5963, 0.1723, 0.4463,
        0.5725, 0.4658, 0.4502, 0.1790, 0.1581, 0.6610, 0.4661],
       device='cuda:0')


Evaluate:  98%|█████████▊| 50/51 [00:16<00:00,  3.17it/s]

logitstensor([0.5639, 0.1771, 0.1645, 0.4092, 0.5478, 0.5223, 0.3212, 0.2058, 0.5226,
        0.2097, 0.4607, 0.4941, 0.4148, 0.5115, 0.4144, 0.2357],
       device='cuda:0')


Evaluate: 100%|██████████| 51/51 [00:16<00:00,  3.06it/s]

logitstensor([0.1724, 0.1718, 0.1793, 0.4311, 0.5655, 0.3298, 0.3922, 0.1662, 0.1617,
        0.6331, 0.2885, 0.4502, 0.3055, 0.2608], device='cuda:0')


In [ ]:
# img_encoder =[vit_encoder,resnet_encoder,swintransformer]
# model = CrossModel(img_encoder,encoder_layer,text_embedding, pooling_layer)

# lr = 0.001
# model = model.to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# epoch = 0
# loss = 10000
# # index_checkpoint = '3'
# # PATH = os.path.join(checkpoint_directory, index_checkpoint + CHECKPOINT_EXTENSION)
# # epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)
# # print("Load checkpoint: "+str(PATH))

# history_losses = train(dataloader_train,model,device ,epochs=100, optimizer=optimizer, cur_epoch=epoch, best_valid_loss=loss)

## Predict

In [ ]:
# def predict(model,device,dataloader,image_processor):
#   list = []
#   model.eval()

#   data_iterator = tqdm(dataloader, desc=f'Epoch')
#   with torch.no_grad():
#     for data in data_iterator:
#       img = torch.squeeze(data[0]['pixel_values'],1)
#       text = torch.squeeze(data[1]['input_ids'],1)
#       target = torch.squeeze(data[2])
#       # print(text.shape)
#       img = img.to(device)
#       text = text.to(device)
#       target = target.to(device).to(torch.int64)
#       optimizer.zero_grad()
      
#       logits = model(img,text)

#       print("logits"+str(torch.sigmoid(logits)))
#       pred = torch.round(torch.sigmoid(logits)).detach().cpu().numpy()
#       list.extend(pred)
#   return list

In [ ]:
# dataset_test = Dataset_MMM(test_file_path, test_img_dir,image_processor,tokenizer,is_url=True)
# dataloader_test = DataLoader(dataset_test,batch_size=16,drop_last=False, shuffle=False)

In [ ]:
# img_encoder =[vit_encoder,resnet_encoder,swintransformer]
# model = CrossModel(img_encoder,encoder_layer,text_embedding, pooling_layer)

# lr=0.001
# model = model.to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# index_checkpoint = '19'
# PATH = os.path.join(checkpoint_directory, index_checkpoint + CHECKPOINT_EXTENSION)
# epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)
# print("Load checkpoint: "+str(PATH))

# list = predict(model,device,dataloader_test,image_processor)
# json_dict = read_json(test_file_path)
# for i in range(len(json_dict)):
#     if list[i] == 0:
#         json_dict[i]['subtask1_label'] = 'NO'
#     else:
#         json_dict[i]['subtask1_label'] = 'YES'
# with open(test_save_path, 'w') as f:
#     json.dump(json_dict, f, indent=4)

In [ ]:
len(json_dict)

1